<a href="https://colab.research.google.com/github/Ayasa18/ProgresBelajar/blob/main/Submission_Akhir_KlasifikasiGambar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Klasifikasi Gambar: [Input Nama Dataset]
- **Nama:** [Aditiya Saputra]
- **Email:** [AditiyaS1811@gmail.com]
- **ID Dicoding:** [aditiya18]

## Import Semua Packages/Library yang Digunakan

In [18]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
from PIL import Image
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback

## Data Preparation

### Data Loading

In [2]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d gunavenkatdoddi/eye-diseases-classification/

Dataset URL: https://www.kaggle.com/datasets/gunavenkatdoddi/eye-diseases-classification/versions/
License(s): ODbL-1.0
 98% 721M/736M [00:03<00:00, 255MB/s]
100% 736M/736M [00:03<00:00, 233MB/s]


# Read Dataset

In [3]:
with zipfile.ZipFile('eye-diseases-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('eye-diseases-classification')

def print_images_resolution(directory):
    """
    Mencetak jumlah gambar di setiap subdirektori dan resolusi unik setiap gambar.
    Args:
        directory (str): Path folder dataset.
    """
    unique_sizes = set()
    total_images = 0

    for subdir in os.listdir(directory):
        subdir_path = os.path.join(directory, subdir)
        if not os.path.isdir(subdir_path):
            continue  # Skip jika bukan direktori

        image_files = os.listdir(subdir_path)
        num_images = len(image_files)
        print(f"{subdir}: {num_images} images")
        total_images += num_images

        for img_file in image_files:
            img_path = os.path.join(subdir_path, img_file)
            try:
                with Image.open(img_path) as img:
                    unique_sizes.add(img.size)
            except Exception as e:
                print(f"Error opening image {img_file}: {e}")

        print("Unique resolutions:")
        for size in unique_sizes:
            print(f"- {size}")
        print("---------------")
        unique_sizes.clear()  # Reset untuk subdir berikutnya

    print(f"\nTotal images: {total_images}")

dataset_folder = "/content/eye-diseases-classification/dataset"
print_images_resolution(dataset_folder)

cataract: 1038 images
Unique resolutions:
- (512, 512)
- (2464, 1632)
- (2592, 1728)
- (1848, 1224)
- (256, 256)
---------------
glaucoma: 1007 images
Unique resolutions:
- (512, 512)
- (2464, 1632)
- (2592, 1728)
- (1848, 1224)
- (256, 256)
---------------
diabetic_retinopathy: 1098 images
Unique resolutions:
- (512, 512)
---------------
normal: 1074 images
Unique resolutions:
- (512, 512)
---------------

Total images: 4217


### Data Preprocessing

#### Split Dataset

In [4]:

# Direktori dataset
dataset_dir = "/content/eye-diseases-classification/dataset/"

# List tiap class
classes = ["cataract", "diabetic_retinopathy", "glaucoma", "normal"]

# Rasio train dan test sets
train_ratio = 0.8

# Loop untuk setiap class
for class_name in classes:
    # Path ke direktori class saat ini
    class_dir = os.path.join(dataset_dir, class_name)

    # List semua file dalam direktori class
    files = os.listdir(class_dir)

    # Split files menjadi train (80%) dan test (20%)
    train_files, test_files = train_test_split(files, train_size=train_ratio, random_state=42)

    # Membuat direktori baru untuk train dan test sets
    for folder_name, file_list in [("train", train_files), ("test", test_files)]:
        folder_path = os.path.join(dataset_dir, folder_name, class_name)
        os.makedirs(folder_path, exist_ok=True)

        # Memindahkan file ke direktori yang sesuai
        for file in file_list:
            src = os.path.join(class_dir, file)
            dst = os.path.join(folder_path, file)
            shutil.move(src, dst)

print("Dataset berhasil dibagi menjadi 80% train dan 20% test.")


Dataset berhasil dibagi menjadi 80% train dan 20% test.


# Data Normalisasi dan Augmentasi

In [25]:
# Dimensi sebuah gambar
img_width, img_height = 256, 256

# Ukuran batch image
batch_size = 64
# Data augmentation dan normalisasi untuk train dan test
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values to [0,1]
)

test_datagen = ImageDataGenerator(
    rescale=1.0 / 255  # Normalize pixel values to [0,1]
)

# Data generators
train_generator = train_datagen.flow_from_directory(
    directory=os.path.join(dataset_dir, 'train'),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    directory=os.path.join(dataset_dir, 'test'),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  #
)

print(f"Train samples: {train_generator.samples}")
print(f"Test samples: {test_generator.samples}")


Found 3372 images belonging to 4 classes.
Found 845 images belonging to 4 classes.
Train samples: 3372
Test samples: 845


## Modelling

In [28]:
num_classes = len(["cataract", "diabetic_retinopathy", "glaucoma", "normal"])

# Membuat model Sequential
model = Sequential()

# Layer Convolutional pertama dengan MaxPooling
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer Convolutional kedua dengan MaxPooling
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer Convolutional ketiga dengan MaxPooling
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten layer untuk mengubah data ke dimensi 1D
model.add(Flatten())

# Fully connected layer (Dense) dengan dropout
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.6))

# Output layer dengan softmax
model.add(Dense(num_classes, activation='softmax'))

# Kompilasi model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Menampilkan arsitektur model
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_22 (Conv2D)                   │ (None, 254, 254, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_21 (MaxPooling2D)      │ (None, 127, 127, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_23 (Conv2D)                   │ (None, 125, 125, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_22 (MaxPooling2D)      │ (None, 62, 62, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_24 (Conv2D)                   │ (None, 60, 60, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_23 (MaxPooling2D)      │ (None, 30, 30, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 230400)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 256)                 │      58,982,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 4)                   │           1,028 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 59,354,500 (226.42 MB)

 Trainable params: 59,354,500 (226.42 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
class CustomStopCallback(Callback):
    def __init__(self, threshold=0.95):
        super(CustomStopCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        train_acc = logs.get('accuracy')
        val_acc = logs.get('val_accuracy')
        if train_acc > self.threshold and val_acc > self.threshold:
            print(f"\nAkurasi sudah mencapai {self.threshold * 100:.2f}%! Menghentikan pelatihan.")
            self.model.stop_training = True

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    min_delta=0.01,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

custom_stop_callback = CustomStopCallback(threshold=0.95)

# Latih Model dengan Callbacks
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    callbacks=[early_stopping, model_checkpoint, custom_stop_callback]
)

Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step - accuracy: 0.3876 - loss: 1.9006
Epoch 1: val_accuracy improved from -inf to 0.62249, saving model to best_model.keras
53/53 ━━━━━━━━━━━━━━━━━━━━ 50s 815ms/step - accuracy: 0.3898 - loss: 1.8886 - val_accuracy: 0.6225 - val_loss: 0.8166
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step - accuracy: 0.6491 - loss: 0.7988
Epoch 2: val_accuracy improved from 0.62249 to 0.76450, saving model to best_model.keras
53/53 ━━━━━━━━━━━━━━━━━━━━ 48s 864ms/step - accuracy: 0.6496 - loss: 0.7978 - val_accuracy: 0.7645 - val_loss: 0.6314
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step - accuracy: 0.7179 - loss: 0.6723
Epoch 3: val_accuracy did not improve from 0.76450
53/53 ━━━━━━━━━━━━━━━━━━━━ 66s 556ms/step - accuracy: 0.7180 - loss: 0.6721 - val_accuracy: 0.7633 - val_loss: 0.5597
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 0.7628 - loss: 0.5778
Epoch 4: val_accuracy improved from 0.76450 to 0.77160, saving model to best

## Evaluasi dan Visualisasi

## Konversi Model

## Inference (Optional)